In [1]:
import pickle
import torch
from torch import nn
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from Dependencies.Adaptive_GCN_GRU_Model import Main_Model as Model
from Dependencies.Model_Training_Module import train_test_datasets, train_loop, test_loop

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
root_path =  'D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/'
file_path_openneuro = 'openneuro_Data/'
file_path = root_path+file_path_openneuro
path = file_path+'TrainingData_Pickle/'+'training_data.p'

In [4]:
file = open( path, "rb" )
dataSet = pickle.load(file)
len(dataSet)

38223

In [5]:
batch_size = 40
train_test_split = 0.7

In [6]:
train_dataloader, test_dataloader = train_test_datasets(dataSet, batch_size, train_test_split)

len(train_dataloader), len(test_dataloader), len(train_dataloader.dataset)+len(test_dataloader.dataset), (len(train_dataloader)-1)*batch_size+len(test_dataloader)

# train_features_signal, train_features_adjMatrix, train_labels = next(iter(train_dataloader))
# test_features_signal, test_features_adjMatrix, test_labels = next(iter(test_dataloader))

# train_features_signal.shape, train_features_adjMatrix.shape, train_labels.shape, test_features_signal.shape, test_features_adjMatrix.shape, test_labels.shape

(669, 287, 38223, 27007)

In [7]:
signal, Adj_dist_matrix, label = dataSet[0]
signal.shape, Adj_dist_matrix.shape, label, len(dataSet)

((3, 19, 400), (19, 19), 0, 38223)

In [8]:
learning_rate = 2e-4
batch_size = 40
epochs = 5

timeSteps, nodesDim, nodeFeaturesDim = signal.shape
hidden_dim = 64           #  
dropout_prob = 0.7        #      
num_layers = 2

model = Model(nodesDim, nodeFeaturesDim, hidden_dim, num_layers, dropout_prob, timeSteps, batch_size)
# outputs = model(train_features_signal, train_features_adjMatrix)
# outputs

In [9]:
loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# steps = len(train_dataloader)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

In [10]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, batch_size)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.171920  [   40/26756]
loss: 0.973574  [ 4040/26756]
loss: 0.630068  [ 8040/26756]
loss: 0.500799  [12040/26756]
loss: 0.677769  [16040/26756]
loss: 0.614692  [20040/26756]
loss: 0.668381  [24040/26756]
Test Error: 
 Accuracy: 65.0%, Avg loss: 0.673460 

Epoch 2
-------------------------------
loss: 0.523545  [   40/26756]
loss: 0.640792  [ 4040/26756]
loss: 0.671637  [ 8040/26756]
loss: 0.681197  [12040/26756]
loss: 0.698648  [16040/26756]
loss: 0.667068  [20040/26756]
loss: 0.643708  [24040/26756]
Test Error: 
 Accuracy: 65.0%, Avg loss: 0.675444 

Epoch 3
-------------------------------
loss: 0.665506  [   40/26756]
loss: 0.617794  [ 4040/26756]
loss: 0.634608  [ 8040/26756]
loss: 0.697788  [12040/26756]
loss: 0.699981  [16040/26756]
loss: 0.647543  [20040/26756]
loss: 0.722403  [24040/26756]
Test Error: 
 Accuracy: 65.0%, Avg loss: 0.669304 

Epoch 4
-------------------------------
loss: 0.648737  [   40/26756]
loss: 0.692777  [ 4040/2

In [11]:
path =  'D:/Workspace/DepressionDetection_FreeThesis/Local_Development_Branch/DepressionDetection/Model_Training/Trained_Models/'
pickle.dump(model, open(path+'model.p', "wb" ))

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------